In [1]:
#import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=100)
lngs = np.random.uniform(low=-180.000, high=180.000, size=100)
lat_lngs = zip(lats, lngs)

# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

87

In [5]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# define url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Loop through all the cities in our list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | fortuna
Processing Record 2 of Set 1 | innisfail
Processing Record 3 of Set 1 | pisco
Processing Record 4 of Set 1 | vaitupu
City not found. Skipping...
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | bubaque
Processing Record 7 of Set 1 | lakatoro
Processing Record 8 of Set 1 | fairbanks
Processing Record 9 of Set 1 | carnarvon
Processing Record 10 of Set 1 | cockburn town
Processing Record 11 of Set 1 | sistranda
Processing Record 12 of Set 1 | pevek
Processing Record 13 of Set 1 | caravelas
Processing Record 14 of Set 1 | raga
Processing Record 15 of Set 1 | mahebourg
Processing Record 16 of Set 1 | albany
Processing Record 17 of Set 1 | ribeira grande
Processing Record 18 of Set 1 | kavaratti
Processing Record 19 of Set 1 | east london
Processing Record 20 of Set 1 | diffa
Processing Record 21 of Set 1 | mount darwin
Processing Record 22 of Set 1 | winsum
Processing Reco

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

# reorder columns
new_column_order = ['City','Country','Date','Lat','Lng','Max Temp',
                    'Humidity','Cloudiness','Wind Speed']
city_data_df = city_data_df[new_column_order]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Fortuna,US,2022-07-18 02:22:32,40.5982,-124.1573,65.93,71,1,5.99
1,Innisfail,AU,2022-07-18 02:27:09,-17.5333,146.0333,71.37,76,94,9.89
2,Pisco,PE,2022-07-18 02:27:09,-13.7000,-76.2167,60.03,88,5,6.91
3,Qaanaaq,GL,2022-07-18 02:27:10,77.4840,-69.3632,44.15,66,92,2.53
4,Bubaque,GW,2022-07-18 02:27:10,11.2833,-15.8333,80.29,84,100,11.95
...,...,...,...,...,...,...,...,...,...
76,Piacabucu,BR,2022-07-18 02:28:33,-10.4056,-36.4344,72.05,81,10,5.55
77,Callaguip,PH,2022-07-18 02:28:34,18.0714,120.4867,86.02,84,75,5.75
78,Saint-Louis,SN,2022-07-18 02:28:34,16.3333,-15.0000,77.94,70,43,13.44
79,Ponta Do Sol,PT,2022-07-18 02:24:23,32.6667,-17.1000,71.01,81,15,4.65


In [9]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")